In [4]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine

In [7]:
conn_string = "DRIVER={ODBC Driver 17 for SQL Server};Server=localhost,1435;Uid=sa;Pwd=amdaripassword01"
conn = pyodbc.connect(conn_string)

In [8]:
cursor = conn.cursor()

In [9]:
conn.autocommit = True

In [11]:
cursor.execute("SELECT name FROM sys.databases;")

In [12]:
cursor.fetchall()

[('master',), ('tempdb',), ('model',), ('msdb',)]

In [13]:
cursor.execute("CREATE DATABASE amdaridb;")

In [14]:
cursor.execute("USE amdaridb;")

In [15]:
create_query_file = open('./sql/sql_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university (\n    id BIGINT IDENTITY(1,1) PRIMARY KEY,\n    country VARCHAR(255) NOT NULL,\n    name VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    type VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n\n    CONSTRAINT unique_combination UNIQUE (country, name)\n)'

In [16]:
cursor.execute(create_query)

In [17]:
engine = create_engine('mssql+pyodbc://', creator=lambda: conn)

In [18]:
pd.read_sql('SELECT * FROM university;', con=engine)

,id,country,name,founded,type,enrollment,link


In [19]:
df = pd.read_csv('universities_cleaned.csv')

In [20]:
df.to_sql('#staging', index=False, con=engine)

70

In [21]:
pd.read_sql("SELECT * FROM #staging WHERE country = 'France';", con=engine)

,Country,University,Founded,Type,Enrollment,Link
0,France,Centre national d'enseignement à distance,1939,Public,350000,https://en.wikipedia.org/wiki/Centre_national_...


In [27]:
merge_query_file = open('./sql/sqlserver_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'MERGE INTO university target\nUSING #staging source\nON (source.university = target.name AND source.Country = target.country)\nWHEN MATCHED THEN UPDATE\nSET\n    target.founded = source.Founded,\n    target.type = source.Type,\n    target.enrollment = source.Enrollment,\n    target.link = source.Link\nWHEN NOT MATCHED THEN\nINSERT (country, name, founded, type, enrollment, link)\nVALUES (source.Country, source.University, source.Founded, source.Type, source.Enrollment, source.Link);'

In [28]:
cursor.execute(merge_query)